Basic Linear AutoEncoder flow using JAADOpenPose dataset:
=========================================================

In [ ]:
# seeding

import pytorch_lightning as pl
import randomname

# get random version name before seeding
version = randomname.get_name()
seed = 22742

pl.seed_everything(seed, workers=True)
print(version)

Model to use:
-------------

In [ ]:
from pedestrians_video_2_carla.data.openpose.skeleton import BODY_25_SKELETON
from pedestrians_video_2_carla.modules.movements.linear_ae.linear_ae_2d import LinearAE2D

model = LinearAE2D(
    input_nodes = BODY_25_SKELETON,
    output_nodes = BODY_25_SKELETON,
    disable_lr_scheduler = True,
    lr = 1e-3,
    model_scaling_factor = 8
)

Flow definition:
----------------

Also handles checkpoint loading if needed.

In [ ]:
# ckpt_path='/runs/JAADOpenPoseDataModule/jaad_autoencoder_notebook/LinearAE2D/other-style/checkpoints/epoch=273-step=8493.ckpt'
ckpt_path = None

In [ ]:
# get flow model (LitAutoencoderFlow)

from pedestrians_video_2_carla.modules.flow.autoencoder import LitAutoencoderFlow
from pedestrians_video_2_carla.data.base.base_transforms import BaseTransforms
from pedestrians_video_2_carla.loss import LossModes

if ckpt_path is not None:
    flow = LitAutoencoderFlow.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        movements_model=model,
    )
else:
    flow = LitAutoencoderFlow(
        movements_model=model,
        transform=BaseTransforms.hips_neck_bbox,
        loss_modes=[LossModes.loc_2d],
    )

In [ ]:
# DataModule to use

from pedestrians_video_2_carla.data.openpose.jaad_openpose_datamodule import JAADOpenPoseDataModule

dm = JAADOpenPoseDataModule(
    batch_size=256,
    clip_offset=1,
    clip_length=1,
    transforms=BaseTransforms.hips_neck_bbox,
    input_nodes=BODY_25_SKELETON,
    skip_metadata=True,
    return_graph=model.needs_graph,
    return_confidence=model.needs_confidence,
    strong_points=True,
    missing_point_probability=0.1,
    noise='gaussian',
    num_workers=0
)

In [ ]:
# loggers and callbacks

import os

from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

from pedestrians_video_2_carla.loggers.pedestrian import PedestrianLogger
from pedestrians_video_2_carla.loggers.pedestrian.enums import PedestrianRenderers

logger = TensorBoardLogger(
    save_dir='/runs',
    name=os.path.join(
        dm.__class__.__name__,
        'jaad_autoencoder_notebook',
        model.__class__.__name__,
    ),
    version=version,
    default_hp_metric=False,
)
pedestrian_logger = PedestrianLogger(
    save_dir=os.path.join(logger.log_dir, "videos"),
    name=logger.name,
    version=logger.version,
    movement_model_output_type=model.output_type,
    # renderers=[PedestrianRenderers.source_videos, PedestrianRenderers.target_points, PedestrianRenderers.input_points, PedestrianRenderers.projection_points],
    renderers=[PedestrianRenderers.none],
    source_videos_dir='/datasets/JAAD/videos',
    source_videos_overlay_skeletons=True,
    video_saving_frequency_reduction=1
)
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(logger.log_dir, "checkpoints"),
    monitor="val_loss/primary",
    mode="min",
    save_top_k=1,
)
lr_monitor = LearningRateMonitor(logging_interval="step")

In [ ]:
# actual trainer

trainer = pl.Trainer(
    # gpus=1,
    # auto_select_gpus=True,
    log_every_n_steps=1,
    num_sanity_val_steps=1,
    check_val_every_n_epoch=1,
    max_epochs=10,
    deterministic=False,
    logger=[logger, pedestrian_logger],
    callbacks=[checkpoint_callback, lr_monitor],
)

In [ ]:
# train!

trainer.fit(model=flow, datamodule=dm, ckpt_path=ckpt_path)  # ckpt_path is needed to resume optimizer states etc.